In [ ]:
from pathlib import Path
import pickle

import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt 

from keras import layers, models, callbacks
from keras import optimizers, regularizers
from keras import losses, metrics
from keras.utils import plot_model
from keras.utils import serialize_keras_object, to_categorical

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoLocator, FixedLocator
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_auc_score

